# Tâche 3 : Détection de Fraude OCR dans les Documents d'Identité

Ce notebook a pour objectif de résoudre la troisième tâche du challenge ANIP : détecter les fraudes dans les documents d'identité en utilisant des techniques d'OCR et de vision par ordinateur.

## Objectif

Développer une IA capable de distinguer automatiquement les documents d'identité authentiques des documents falsifiés, en analysant :
- Les incohérences visuelles
- Les anomalies textuelles via OCR
- Les patterns de falsification

## Types de Documents Analysés

Notre dataset contient des documents de différents pays/types :
- **Arizona DL** : Permis de conduire d'Arizona
- **ESP** : Documents espagnols  
- **EST** : Documents estoniens
- **RUS** : Documents russes

## Stratégie Adoptée

Notre approche sera méthodique et multi-modale :

1. **Analyse Exploratoire des Données** : Comprendre les types de fraudes et leur distribution
2. **Préparation des Données** : Organisation par type de document et de fraude
3. **Extraction de Features** : OCR + Features visuelles + Analyse de qualité d'image
4. **Modélisation Multi-Classe** : Classification binaire et détection du type de fraude
5. **Évaluation Robuste** : Métriques adaptées à la sécurité (précision sur les fraudes)
6. **Pipeline de Production** : Système de détection en temps réel

In [ ]:
# Importation des bibliothèques fondamentales
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
import cv2
from collections import Counter, defaultdict
import warnings
warnings.filterwarnings('ignore')

# Bibliothèques pour OCR et traitement d'image
try:
    import pytesseract
    print("✅ Tesseract OCR disponible")
except ImportError:
    print("⚠️  Tesseract OCR non installé - installation nécessaire")

# Bibliothèques ML/DL
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models

# Configuration des graphiques
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")
plt.rcParams['figure.figsize'] = (12, 8)

# Configuration du device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Device utilisé: {device}")
print("✅ Bibliothèques de base importées.")

In [ ]:
# Configuration des chemins de données
from pathlib import Path
import os

# Chemins vers les données de la tâche 3
DATA_ROOT = Path('/workspaces/anip-facial-ocr-challenge/dataset_tache_3/dataset_tache_3')
TRAIN_DIR = DATA_ROOT / 'train'
SUBMISSIONS_DIR = Path('/workspaces/anip-facial-ocr-challenge/submissions')

print("🎯 Configuration des chemins:")
print(f"   📁 Dataset root: {DATA_ROOT}")
print(f"   🚂 Train directory: {TRAIN_DIR}")
print(f"   📤 Submissions: {SUBMISSIONS_DIR}")

# Vérification de l'existence des données
train_exists = TRAIN_DIR.exists()
print(f"\n🔍 Vérification:")
print(f"   Train folder: {'✅' if train_exists else '❌'} {TRAIN_DIR}")

if train_exists:
    # Explorer la structure des données
    document_types = [d for d in TRAIN_DIR.iterdir() if d.is_dir()]
    print(f"\n📊 Types de documents trouvés: {len(document_types)}")
    
    for doc_type in document_types:
        print(f"\n📋 {doc_type.name.upper()}:")
        subdirs = [d for d in doc_type.iterdir() if d.is_dir()]
        
        for subdir in subdirs:
            # Compter les images dans chaque sous-dossier
            image_files = []
            for ext in ['*.jpg', '*.JPG', '*.jpeg', '*.JPEG', '*.png', '*.PNG']:
                image_files.extend(list(subdir.glob(ext)))
            
            print(f"   📁 {subdir.name}: {len(image_files)} images")
            
            # Afficher quelques exemples de noms de fichiers
            if len(image_files) > 0:
                print(f"      🔎 Exemples: {', '.join([f.name for f in image_files[:3]])}")
                if len(image_files) > 3:
                    print(f"         ... et {len(image_files) - 3} autres")
else:
    print("   ⚠️  Les données de la tâche 3 ne sont pas disponibles!")
    print("   💡 Assure-toi que le dataset est bien placé dans le bon répertoire")

In [ ]:
# Création d'un DataFrame complet avec toutes les images et leurs labels
def create_fraud_dataset():
    """
    Crée un DataFrame avec toutes les images, leurs chemins et labels
    """
    data_list = []
    
    if not TRAIN_DIR.exists():
        print("❌ Données non disponibles")
        return pd.DataFrame()
    
    document_types = [d for d in TRAIN_DIR.iterdir() if d.is_dir()]
    
    for doc_type in document_types:
        doc_name = doc_type.name
        subdirs = [d for d in doc_type.iterdir() if d.is_dir()]
        
        for subdir in subdirs:
            category = subdir.name
            
            # Déterminer le label de fraude
            if category == 'normal':
                fraud_label = 'authentic'
                fraud_type = 'none'
            elif category == 'gt':
                fraud_label = 'ground_truth'
                fraud_type = 'reference'
            elif category.startswith('forgery_'):
                fraud_label = 'fraud'
                fraud_type = category
            else:
                fraud_label = 'unknown'
                fraud_type = category
            
            # Récupérer toutes les images
            image_files = []
            for ext in ['*.jpg', '*.JPG', '*.jpeg', '*.JPEG', '*.png', '*.PNG']:
                image_files.extend(list(subdir.glob(ext)))
            
            # Ajouter chaque image au dataset
            for img_path in image_files:
                data_list.append({
                    'file_path': str(img_path),
                    'filename': img_path.name,
                    'document_type': doc_name,
                    'category': category,
                    'fraud_label': fraud_label,
                    'fraud_type': fraud_type,
                    'file_size': img_path.stat().st_size if img_path.exists() else 0
                })
    
    return pd.DataFrame(data_list)

# Créer le dataset principal
print("🔍 CRÉATION DU DATASET DE FRAUDE DOCUMENTAIRE")
print("="*50)

df_fraud = create_fraud_dataset()

if len(df_fraud) > 0:
    print(f"✅ Dataset créé avec succès!")
    print(f"   Total d'images: {len(df_fraud):,}")
    print(f"   Types de documents: {df_fraud['document_type'].nunique()}")
    print(f"   Catégories: {df_fraud['category'].nunique()}")
    
    print("\n📊 Aperçu du dataset:")
    print(df_fraud.head(10))
    
    print(f"\n📋 Informations sur le dataset:")
    print(df_fraud.info())
    
else:
    print("❌ Impossible de créer le dataset - données non disponibles")

## 2. Analyse Exploratoire des Données de Fraude

Maintenant que nous avons exploré la structure, analysons en détail les différents types de fraudes et leurs caractéristiques.

## 2. Analyse Exploratoire des Données de Fraude

Maintenant que nous avons exploré la structure, analysons en détail les différents types de fraudes et leurs caractéristiques.